In [3]:
import pandas as pd


invoice       = pd.read_parquet("dataset_creados\\invoice.parquet")
invoice_line  = pd.read_parquet("dataset_creados\\invoiceLine.parquet")
partner       = pd.read_parquet("dataset_creados\\partner.parquet")
product       = pd.read_parquet("dataset_creados\\product.parquet")
payment       = pd.read_parquet("dataset_creados\\payment.parquet")
payment_app   = pd.read_parquet("dataset_creados\\paymentApplication.parquet")




In [1]:
def test_result(nombre, condicion):
    estado = "PASS" if condicion else "FAIL"
    print(f"{nombre:<60} {estado}")


In [4]:
tests = []

tests.append(("Invoice: columnas requeridas",
    set(["id", "invoice_date", "due_date", "partner_id", "invoice_type", "total_amount", "remaining_balance"]).issubset(invoice.columns)
))

tests.append(("InvoiceLine: columnas requeridas",
    set(["id", "invoice_id", "product_id", "quantity", "price"]).issubset(invoice_line.columns)
))

tests.append(("Product: columnas requeridas",
    set(["id", "product_name", "cost_per_unit"]).issubset(product.columns)
))

tests.append(("Partner: columnas requeridas",
    set(["id", "display_name"]).issubset(partner.columns)
))

tests.append(("Payment: columnas requeridas",
    set(["id", "payment_date", "amount"]).issubset(payment.columns)
))

tests.append(("PaymentApplication: columnas requeridas",
    set(["payment_id", "invoice_id", "allocated_amount"]).issubset(payment_app.columns)
))

for nombre, condicion in tests:
    test_result(nombre, condicion)


Invoice: columnas requeridas                                 PASS
InvoiceLine: columnas requeridas                             PASS
Product: columnas requeridas                                 PASS
Partner: columnas requeridas                                 PASS
Payment: columnas requeridas                                 PASS
PaymentApplication: columnas requeridas                      PASS


In [5]:

#Integridad referencial
tests = []

tests.append(("InvoiceLine → Invoice",
    invoice_line["invoice_id"].isin(invoice["id"]).all()
))

tests.append(("InvoiceLine → Product",
    invoice_line["product_id"].isin(product["id"]).all()
))

tests.append(("Invoice → Partner",
    invoice["partner_id"].isin(partner["id"]).all()
))

tests.append(("PaymentApplication → Payment",
    payment_app["payment_id"].isin(payment["id"]).all()
))

tests.append(("PaymentApplication → Invoice",
    payment_app["invoice_id"].isin(invoice["id"]).all()
))

for nombre, condicion in tests:
    test_result(nombre, condicion)

InvoiceLine → Invoice                                        PASS
InvoiceLine → Product                                        PASS
Invoice → Partner                                            PASS
PaymentApplication → Payment                                 PASS
PaymentApplication → Invoice                                 PASS


In [6]:
tests = []

tests.append(("Invoice: total_amount > 0",
    (invoice["total_amount"] > 0).all()
))

tests.append(("InvoiceLine: quantity > 0",
    (invoice_line["quantity"] > 0).all()
))

tests.append(("InvoiceLine: price ≥ 0",
    (invoice_line["price"] >= 0).all()
))

tests.append(("Invoice: remaining_balance ≤ total_amount",
    (invoice["remaining_balance"] <= invoice["total_amount"]).all()
))

tests.append(("Invoice: due_date ≥ invoice_date",
    (invoice["due_date"] >= invoice["invoice_date"]).all()
))

tests.append(("Payment: amount > 0",
    (payment["amount"] > 0).all()
))

tests.append(("PaymentApp: allocated_amount ≤ payment.amount",
    payment_app["allocated_amount"].le(
        payment_app.merge(payment, left_on="payment_id", right_on="id")["amount"]
    ).all()
))

for nombre, condicion in tests:
    test_result(nombre, condicion)


Invoice: total_amount > 0                                    PASS
InvoiceLine: quantity > 0                                    PASS
InvoiceLine: price ≥ 0                                       PASS
Invoice: remaining_balance ≤ total_amount                    PASS
Invoice: due_date ≥ invoice_date                             FAIL
Payment: amount > 0                                          PASS
PaymentApp: allocated_amount ≤ payment.amount                PASS


In [7]:
errores = {}


errores["remaining_balance > total_amount"] = invoice[invoice["remaining_balance"] > invoice["total_amount"]]

# allocated_amount mayor al payment.amount
errores["allocated_amount > payment.amount"] = payment_app[
    payment_app["allocated_amount"] >
    payment_app.merge(payment, left_on="payment_id", right_on="id")["amount"]
]

# Fechas invertidas
errores["due_date < invoice_date"] = invoice[invoice["due_date"] < invoice["invoice_date"]]

# Product inexistente
errores["product_id inexistente"] = invoice_line[~invoice_line["product_id"].isin(product["id"])]

# Partner inexistente
errores["partner_id inexistente"] = invoice[~invoice["partner_id"].isin(partner["id"])]

# Facturas sin líneas
errores["Factura sin líneas"] = invoice[~invoice["id"].isin(invoice_line["invoice_id"])]

# Mostrar PASS/FAIL
for nombre, df_err in errores.items():
    condicion = len(df_err) == 0
    test_result(f"Error intencional: {nombre}", condicion)

    if len(df_err) > 0:
        print("\nDetalles del error:")
        display(df_err.head())


Error intencional: remaining_balance > total_amount          PASS
Error intencional: allocated_amount > payment.amount         PASS
Error intencional: due_date < invoice_date                   FAIL

Detalles del error:


,id,partner_id,currency_id,invoice_type,invoice_date,due_date,total_amount,invoice_id,amount_paid,remaining_balance,payment_state
0,1,39,1,in,2025-02-12 10:29:00.900991,2025-02-07 10:29:00.900991,4324,1.0,93.0,4231.0,partial


Error intencional: product_id inexistente                    PASS
Error intencional: partner_id inexistente                    PASS
Error intencional: Factura sin líneas                        PASS
